imported libraries for data handling, text vectorization, model training, and evaluation.

In [21]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [22]:
df = pd.read_csv("fake_job_postings.csv")

df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [23]:
df['text'] = (
    df['description'].fillna('') + ' ' +
    df['requirements'].fillna('') + ' ' +
    df['company_profile'].fillna('')
)

X = df['text']
y = df['fraudulent']

combined all relevant textual information into a single feature for NLP processing.

In [24]:
def clean_text(text):
    text = text.lower()                     # lowercase
    text = re.sub(r'[^a-z\s]', '', text)    # remove punctuation & numbers
    text = re.sub(r'\s+', ' ', text)        # remove extra spaces
    return text

In [25]:
X = X.apply(clean_text)

Text preprocessing helps remove noise and improve model learning.

In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

splitted the dataset into training and testing sets to evaluate generalization.

In [27]:
vectorizer = TfidfVectorizer(
    stop_words='english',
    max_features=5000
)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

TF-IDF converts text into numerical features based on word importance.

In [28]:
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

MultinomialNB()

Naive Bayes is efficient and performs well for text classification problems.

In [29]:
y_pred = model.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9678411633109619

Confusion Matrix:
 [[3395    0]
 [ 115   66]]

Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      3395
           1       1.00      0.36      0.53       181

    accuracy                           0.97      3576
   macro avg       0.98      0.68      0.76      3576
weighted avg       0.97      0.97      0.96      3576



The model achieves good accuracy in identifying fraudulent job postings.

In [30]:
def predict_job(job_text):
    job_text = clean_text(job_text)
    job_tfidf = vectorizer.transform([job_text])
    prediction = model.predict(job_tfidf)[0]

    if prediction == 1:
        return "⚠️ Scam Job Detected"
    else:
        return "✅ Legitimate Job"

# Example
sample_job = "Earn money from home, no experience needed, quick payment"
print(predict_job(sample_job))

✅ Legitimate Job


In [31]:
print(predict_job(
    "Work from home. Earn money fast. No experience needed. "
    "WhatsApp us immediately. Registration fee required."
))

⚠️ Scam Job Detected


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1,2)
)

X_tfidf = vectorizer.fit_transform(X)  # X = text column

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y, test_size=0.2, random_state=42, stratify=y
)

In [34]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)

In [35]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_bal, y_train_bal)

MultinomialNB()

In [36]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[3118  285]
 [  19  154]]
              precision    recall  f1-score   support

           0       0.99      0.92      0.95      3403
           1       0.35      0.89      0.50       173

    accuracy                           0.91      3576
   macro avg       0.67      0.90      0.73      3576
weighted avg       0.96      0.91      0.93      3576



In [37]:
import pickle

pickle.dump(model, open("job_scam_model.pkl", "wb"))
pickle.dump(vectorizer, open("tfidf_vectorizer.pkl", "wb"))

The trained model is saved for deployment.